Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas


In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=25&fecha1_anomes=2019-10&fecha2_dia=12&fecha2_anomes=2019-11&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=25/10/2019&vuelos_fecha_regreso_ddmmaaaa=12/11/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='
r = requests.get(url)

In [3]:
r.status_code

200

In [4]:
s = BeautifulSoup(r.text, 'lxml')
print(s.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   Selecciona tus vuelos | LATAM Airlines
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://s.latamstatic.com/static/latam/images/favicon/LATAM/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <meta content="personas_bs3_latam_mu" name="layout"/>
  <meta content="LAN.com" name="description"/>
  <meta content="LAN.com" name="keywords"/>
  <meta content="F91A0E29822E24335A84243953EE302E" name="sessionId" scheme="tracker"/>
  <meta content="latam" name="matchedScenario" scheme="tracker"/>
  <meta content="personas_bs3_latam_mu" name="template" scheme="tracker"/>
  <meta content="null" name="userDataCookieRawData" scheme="tracker"/>
  <meta content="3.0.53" name="appVersion" scheme="tracker"/>
  <style>
   .async-hide{opacity:0!important}
  

Vemos que la respuesta de la página no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código js que está en la respuesta.

In [5]:
from selenium import webdriver

In [28]:
from tqdm import tqdm

In [7]:
url='https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=25&fecha1_anomes=2019-10&fecha2_dia=12&fecha2_anomes=2019-11&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=25/10/2019&vuelos_fecha_regreso_ddmmaaaa=12/11/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
driver.get(url)

cada vuelo es un `<li class='flight'>` 

In [8]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')

Nos quedamos con un vuelo para analizarlo

In [9]:
vuelo = vuelos[0]

IndexError: list index out of range

#### Hora de llegada, hora de salida y duración

In [ ]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [ ]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
# Hora de salida
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

#### Escalas

La cantidad de escalas aparece en un `<div class='flight-summary-stops-description'>`

In [ ]:
elemento_escalas = vuelo.find_element_by_class_name('flight-summary-stops-description')

Ahora debemos clickear el botón de las escalas para conocer los números de vuelo, la duración de los vuelos y la duración de las escalas

In [ ]:
elemento_escalas.find_element_by_tag_name('button').click()

In [ ]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
segmentos

In [ ]:
escalas = len(segmentos) - 1 #0 escalas si es un vuelo directo

Me quedo con un segmento para analizarlo

In [ ]:
segmento = segmentos[0]

In [ ]:
segmento.get_attribute('class')

In [ ]:
segmento.text

De cada segmento necesitamos:
- origen
- hora de salida
- destino
- hora de llegada
- duración del vuelo
- numero de vuelo
- modelo de avion
- duración de la escala

In [ ]:
# Origen
segmento.find_element_by_xpath('.//div[@class="departure"]/span[@class="ground-point-name"]').text

In [ ]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [ ]:
# Destino
segmento.find_element_by_xpath('.//div[@class="arrival"]/span[@class="ground-point-name"]').text

In [ ]:
# Hora de llegada
segmento.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
# Duración del vuelo
segmento.find_element_by_xpath('.//span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')

In [ ]:
# Numero del vuelo
segmento.find_element_by_xpath('.//span[@class="equipment-airline-number"]').text

In [ ]:
# Modelo de avion
segmento.find_element_by_xpath('.//span[@class="equipment-airline-material"]').text

In [ ]:
# Duracion de la escala
segmento.find_element_by_xpath('.//div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')

Juntamos todo en una función que itere en cada segmento

In [ ]:
info_escalas = []
for segmento in segmentos:
    # Origen
    origen = segmento.find_element_by_xpath(
        './/div[@class="departure"]/span[@class="ground-point-name"]').text
    # Hora de salida
    dep_time = segmento.find_element_by_xpath(
        './/div[@class="departure"]/time').get_attribute('datetime')
    # Destino
    destino = segmento.find_element_by_xpath(
        './/div[@class="arrival"]/span[@class="ground-point-name"]').text
    # Hora de llegada
    arr_time = segmento.find_element_by_xpath(
        './/div[@class="arrival"]/time').get_attribute('datetime')
    # Duración del vuelo
    duracion_vuelo = segmento.find_element_by_xpath(
        './/span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')
    # Numero del vuelo
    numero_vuelo = segmento.find_element_by_xpath(
        './/span[@class="equipment-airline-number"]').text
    # Modelo de avion
    modelo_avion = segmento.find_element_by_xpath(
        './/span[@class="equipment-airline-material"]').text
    # Duracion de la escala
    if segmento != segmentos[-1]:
        duracion_escala = segmento.find_element_by_xpath(
            './/div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')
    else:
        duracion_escala = ''
        
    # Armo un diccionario para almacenar los datos
    data_dict={'origen': origen, 
                'dep_time': dep_time, 
                'destino': destino,
                'arr_time': arr_time,
                'duracion_vuelo': duracion_vuelo,
                'numero_vuelo': numero_vuelo,
                'modelo_avion': modelo_avion,
                'duracion_escala': duracion_escala}
    print(data_dict)
    info_escalas.append(data_dict)

Debemos cerrar el pop-up 

In [ ]:
driver.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()

#### Precios disponibles

In [ ]:
vuelo.click()

La información de los precios para cada tarifa está contenida en una tabla. Los precios en sí están en el footer y podemos sacar los nombres de la clase de cada elemento

In [ ]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

In [ ]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
    dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

## Juntamos todo

In [10]:
def obtener_precios(vuelo):
    tarifas = vuelo.find_elements_by_xpath(
        './/div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
        dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)
    return precios

In [11]:
def obtener_datos_escalas(vuelo):
    segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
    info_escalas = []
    for segmento in segmentos:
        # Origen
        origen = segmento.find_element_by_xpath(
            './/div[@class="departure"]/span[@class="ground-point-name"]').text
        # Hora de salida
        dep_time = segmento.find_element_by_xpath(
            './/div[@class="departure"]/time').get_attribute('datetime')
        # Destino
        destino = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/span[@class="ground-point-name"]').text
        # Hora de llegada
        arr_time = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/time').get_attribute('datetime')
        # Duración del vuelo
        duracion_vuelo = segmento.find_element_by_xpath(
            './/span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')
        # Numero del vuelo
        numero_vuelo = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-number"]').text
        # Modelo de avion
        modelo_avion = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-material"]').text
        # Duracion de la escala
        if segmento != segmentos[-1]:
            duracion_escala = segmento.find_element_by_xpath(
                './/div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')
        else:
            duracion_escala = ''

        # Armo un diccionario para almacenar los datos
        data_dict={'origen': origen, 
                    'dep_time': dep_time, 
                    'destino': destino,
                    'arr_time': arr_time,
                    'duracion_vuelo': duracion_vuelo,
                    'numero_vuelo': numero_vuelo,
                    'modelo_avion': modelo_avion,
                    'duracion_escala': duracion_escala}
        info_escalas.append(data_dict)
        
    return info_escalas

In [12]:
def obtener_tiempos(vuelo):
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    # Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    return {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}

In [13]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print('Iniciando scraping...')
    info = []
    for vuelo in tqdm(vuelos):
        # Obtenemos los tiempos generales del vuelo
        tiempos = obtener_tiempos(vuelo)
        # Clickeamos el botón de escalas para ver los detalles
        vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]//button').click()
        escalas = obtener_datos_escalas(vuelo)
        # Cerramos el pop-up con los detalles
        vuelo.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()
        # Clickeamos el vuelo para ver los precios
        vuelo.click()
        precios = obtener_precios(vuelo)
        info.append({'precios':precios, 'tiempos':tiempos , 'escalas': escalas})
    return info

Agregamos funcionalidades para que espere a que cargue la página

In [14]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [15]:
url='https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=25&fecha1_anomes=2019-10&fecha2_dia=12&fecha2_anomes=2019-11&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=25/10/2019&vuelos_fecha_regreso_ddmmaaaa=12/11/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
driver.get(url)
delay = 10
try:
    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print("Page is ready!")
    info = obtener_info(driver)
except TimeoutException:
    print("Loading took too much time!")
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=77.0.3865.90)


Ahora debemos ver cómo construir la URL del sitio para hacer consultas

In [ ]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=08&fecha1_anomes=2019-10&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=08/10/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='

In [16]:
url_base = 'https://www.latam.com/es_ar/apps/personas/booking?' 

In [ ]:
params = url.strip(url_base).split('&')
params

In [17]:
import time

In [21]:
fecha = '10/12/2019'

In [22]:
fecha = time.strptime(fecha, '%d/%m/%Y')

In [23]:
time.strftime('%d/%m/%Y',fecha)

'10/12/2019'

In [24]:
def armar_url(url_base, fecha, origen, destino, cabina):
    url = url_base
    url+=f'&fecha1_dia={fecha.tm_mday}'
    url+=f'&fecha1_anomes={fecha.tm_year}-{fecha.tm_mon}'
    url+=f'&auAvailability=1'
    url+=f'&ida_vuelta=ida'
    url+=f'&from_city1={origen}'
    url+=f'&to_city1={destino}'
    url+=f'&vuelos_fecha_salida_ddmmaaaa={time.strftime("%d/%m/%Y",fecha)}'
    url+=f'&cabina={cabina}'
    url+=f'&nadults=1'
    return url

In [35]:
url = armar_url(url_base,fecha, 'MAD', 'BUE', 'Y')
url

'https://www.latam.com/es_ar/apps/personas/booking?&fecha1_dia=10&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&from_city1=MAD&to_city1=BUE&vuelos_fecha_salida_ddmmaaaa=10/12/2019&cabina=Y&nadults=1'

In [36]:
def scrape_latam(urls):
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
    delay = 10
    # Si es un string único, lo convierto en lista
    if type(urls) == str:
        urls = [urls]

    print(urls)
    info = []
    for url in urls:
        print('Scraping URL:',url)
        driver.get(url)
        try:
            myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
            print("Page is ready!")
            info.append(obtener_info(driver))
        except TimeoutException:
            print("Loading took too much time!")
    driver.close()
    return info

In [37]:
scrape_latam(url)

['https://www.latam.com/es_ar/apps/personas/booking?&fecha1_dia=10&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&from_city1=MAD&to_city1=BUE&vuelos_fecha_salida_ddmmaaaa=10/12/2019&cabina=Y&nadults=1']
Scraping URL: https://www.latam.com/es_ar/apps/personas/booking?&fecha1_dia=10&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&from_city1=MAD&to_city1=BUE&vuelos_fecha_salida_ddmmaaaa=10/12/2019&cabina=Y&nadults=1


  0%|          | 0/14 [00:00<?, ?it/s]

Page is ready!
Se encontraron 14 vuelos.
Iniciando scraping...


100%|██████████| 14/14 [00:22<00:00,  1.61s/it]


[[{'precios': [{'PLUS': {'moneda': 'US$', 'valor': '1589,10'}},
    {'TOP': {'moneda': 'US$', 'valor': '1754,10'}}],
   'tiempos': {'hora_salida': '22:15',
    'hora_llegada': '09:10',
    'duracion': 'PT14H55M'},
   'escalas': [{'origen': 'MAD',
     'dep_time': '22:15',
     'destino': 'GRU',
     'arr_time': '05:00',
     'duracion_vuelo': 'PT10H45M',
     'numero_vuelo': 'LA8065',
     'modelo_avion': 'Airbus 350-900',
     'duracion_escala': 'PT1H10M'},
    {'origen': 'GRU',
     'dep_time': '06:10',
     'destino': 'EZE',
     'arr_time': '09:10',
     'duracion_vuelo': 'PT3H',
     'numero_vuelo': 'LA8012',
     'modelo_avion': 'Airbus 320-200',
     'duracion_escala': ''}]},
  {'precios': [{'PLUS': {'moneda': 'US$', 'valor': '1589,10'}},
    {'TOP': {'moneda': 'US$', 'valor': '1754,10'}}],
   'tiempos': {'hora_salida': '22:15',
    'hora_llegada': '10:15',
    'duracion': 'PT16H'},
   'escalas': [{'origen': 'MAD',
     'dep_time': '22:15',
     'destino': 'GRU',
     'arr_time'